In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms,datasets


In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
epochs = 40
batch_size= 64

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('../data',train=True, download=True, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,),(0.3081,))
    ])),
    batch_size = batch_size, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('../data',train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,),(0.3081,))
    ])),
    batch_size = batch_size, shuffle=True
)

In [8]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,10,kernel_size=5)
        self.conv2 = nn.Conv2d(10,20, kernel_size=5)
        self.drop = nn.Dropout2d()
        
        self.fc1 = nn.Linear(320,50)
        self.fc2 = nn.Linear(50,10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x),2))
        x= F.relu(F.max_pool2d(self.conv2(x),2))
        x = x.view(-1,320)
        
        x= F.relu(self.fc1(x))
        x = self.drop(x)
        x= self.fc2(x)
        
        return F.log_softmax(x, dim=1)

In [10]:
model = CNN().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)

In [11]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data,target) in enumerate(train_loader):
        data,target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss=F.cross_entropy(output,target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 200 == 0:
            print('Train Epoch : {}, [{}/{} ({:.0f}%)]\t Loss : {:.6f}'.format(epoch,batch_idx*len(data),len(train_loader.dataset),100.*batch_idx/len(train_loader),loss.item()))

In [14]:
def evalauate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data,target in test_loader:
            data,target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output,target,reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_acc = 100. * correct / len(test_loader.dataset)
    return test_loss, test_acc

In [17]:
for epoch in range(1,epochs+1):
    train(model,train_loader, optimizer,epoch)
    test_loss,test_acc = evalauate(model,test_loader)
    
    print('[{}] Test Loss : {:.4f}, Acc : {:.2f}%'.format(epoch,test_loss, test_acc))

Train Epoch : 1, [0/60000 (0%)]	 Loss : 1.071738
Train Epoch : 1, [12800/60000 (21%)]	 Loss : 0.850949
Train Epoch : 1, [25600/60000 (43%)]	 Loss : 1.020160
Train Epoch : 1, [38400/60000 (64%)]	 Loss : 0.927595
Train Epoch : 1, [51200/60000 (85%)]	 Loss : 0.918161
[1] Test Loss : 0.7385, Acc : 72.96%
Train Epoch : 2, [0/60000 (0%)]	 Loss : 0.704000
Train Epoch : 2, [12800/60000 (21%)]	 Loss : 0.971825
Train Epoch : 2, [25600/60000 (43%)]	 Loss : 0.921776
Train Epoch : 2, [38400/60000 (64%)]	 Loss : 0.873138
Train Epoch : 2, [51200/60000 (85%)]	 Loss : 0.809617
[2] Test Loss : 0.7051, Acc : 73.64%
Train Epoch : 3, [0/60000 (0%)]	 Loss : 0.745469
Train Epoch : 3, [12800/60000 (21%)]	 Loss : 0.750270
Train Epoch : 3, [25600/60000 (43%)]	 Loss : 0.706826
Train Epoch : 3, [38400/60000 (64%)]	 Loss : 0.832731
Train Epoch : 3, [51200/60000 (85%)]	 Loss : 0.661748
[3] Test Loss : 0.6808, Acc : 74.86%
Train Epoch : 4, [0/60000 (0%)]	 Loss : 0.848404
Train Epoch : 4, [12800/60000 (21%)]	 Loss : 

KeyboardInterrupt: 